<a href="https://colab.research.google.com/github/karthikkumarcholleti/AI-Assistant/blob/Experimental-addons/FINE_TUNED_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

#This data set consists of various Queries of the users, used for tuning the model
data = pd.read_csv("FINAL_BALANCED_DATASET.csv")


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


intent_labels = {intent: idx for idx, intent in enumerate(data['INTENT'].unique())}
data['label'] = data['INTENT'].map(intent_labels)  # Ensure the label column is present


def tokenize_function(examples):
    return tokenizer(examples['QUERY'], padding="max_length", truncation=True)


dataset = Dataset.from_pandas(data[['QUERY', 'label']])
tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

#BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(intent_labels))

#Training arguments
training_args = TrainingArguments(
    output_dir='./results_intent',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_intent',
    logging_steps=10,
)

#Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

#Fine-tuning
trainer.train()

# Saving the model
model.save_pretrained('./fine_tuned_intent_model')
tokenizer.save_pretrained('./fine_tuned_intent_model')

#Evaluate the model
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.217200
20,1.157500
30,1.141200
40,1.052300
50,0.903300
60,0.822800
70,0.626200
80,0.514900
90,0.393900
100,0.272200


{'eval_loss': 6.328599556582049e-05,
 'eval_runtime': 27.9272,
 'eval_samples_per_second': 33.372,
 'eval_steps_per_second': 4.189,
 'epoch': 3.0}

In [ ]:
model.save_pretrained("./fine_tuned_intent_model1")
tokenizer.save_pretrained("./fine_tuned_intent_model1")

('./fine_tuned_intent_model1/tokenizer_config.json',
 './fine_tuned_intent_model1/special_tokens_map.json',
 './fine_tuned_intent_model1/vocab.txt',
 './fine_tuned_intent_model1/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#fine-tuned Intent model
model.save_pretrained('/content/drive/MyDrive/fine_tuned_intent_model1')
tokenizer.save_pretrained('/content/drive/MyDrive/fine_tuned_intent_model1')


('/content/drive/MyDrive/fine_tuned_intent_model1/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_intent_model1/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_intent_model1/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_intent_model1/added_tokens.json')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

Intent_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/fine_tuned_intent_model1')
Intent_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/fine_tuned_intent_model1')




In [ ]:
# Fine tuning Price_Type

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

#Loading the dataset
data = pd.read_csv("FINAL_BALANCED_DATASET.csv")


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

price_type_labels = {price_type: idx for idx, price_type in enumerate(data['PRICE_TYPE'].unique())}
data['label'] = data['PRICE_TYPE'].map(price_type_labels)

def tokenize_function(examples):
    return tokenizer(examples['QUERY'], padding="max_length", truncation=True)

dataset = Dataset.from_pandas(data[['QUERY', 'label']])
tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# train and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

#BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(price_type_labels))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results_price_type',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_price_type',
    logging_steps=10,
)

#Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

#Fine-tuning
trainer.train()

# Saving the model
model.save_pretrained('./fine_tuned_price_type_model')
tokenizer.save_pretrained('./fine_tuned_price_type_model')

trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/4658 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.891200
20,1.904400
30,1.877900
40,1.851900
50,1.794200
60,1.705100
70,1.640600
80,1.591600
90,1.519700
100,1.393700


{'eval_loss': 0.0003345948352944106,
 'eval_runtime': 28.2997,
 'eval_samples_per_second': 32.933,
 'eval_steps_per_second': 4.134,
 'epoch': 3.0}

In [ ]:

model.save_pretrained('./fine_tuned_price_type_model')
tokenizer.save_pretrained('./fine_tuned_price_type_model')



('./fine_tuned_price_type_model/tokenizer_config.json',
 './fine_tuned_price_type_model/special_tokens_map.json',
 './fine_tuned_price_type_model/vocab.txt',
 './fine_tuned_price_type_model/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

model.save_pretrained('/content/drive/MyDrive/fine_tuned_price_type_model')
tokenizer.save_pretrained('/content/drive/MyDrive/fine_tuned_price_type_model')


('/content/drive/MyDrive/fine_tuned_price_type_model/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_price_type_model/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_price_type_model/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_price_type_model/added_tokens.json')

In [ ]:
intent_labels = {intent: idx for idx, intent in enumerate(data['INTENT'].unique())}
print(intent_labels)

{'price_inquiry': 0, 'stock_comparison': 1, 'stock_trend': 2}


In [ ]:
price_labels = {price_type: idx for idx, price_type in enumerate(data['PRICE_TYPE'].unique())}
print(price_labels)

{'low': 0, 'open': 1, 'high': 2, 'close': 3, 'volume': 4, 'current': 5}


In [ ]:
#Finetuning the model with Time Period

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

#Load the dataset
data = pd.read_csv("FINAL_BALANCED_DATASET.csv")


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


time_labels = {time_period: idx for idx, time_period in enumerate(data['TIME_PERIOD'].unique())}
data['label'] = data['TIME_PERIOD'].map(time_labels)  # Ensure the label column is present


def tokenize_function(examples):
    return tokenizer(examples['QUERY'], padding="max_length", truncation=True)


dataset = Dataset.from_pandas(data[['QUERY', 'label']])
tokenized_dataset = dataset.map(tokenize_function, batched=True)


tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

#train and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(time_labels))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results_time_period',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_time_period',
    logging_steps=10,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

#Fine-tuning
trainer.train()

# Saving the model
model.save_pretrained('./fine_tuned_time_period_model')
tokenizer.save_pretrained('./fine_tuned_time_period_model')

# Evaluate
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/4658 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,2.208000
20,2.194600
30,2.186500
40,2.094100
50,2.055800
60,2.015800
70,1.948000
80,1.931900
90,1.746000
100,1.790000


Step,Training Loss
10,2.208000
20,2.194600
30,2.186500
40,2.094100
50,2.055800
60,2.015800
70,1.948000
80,1.931900
90,1.746000
100,1.790000


{'eval_loss': 0.0005373093881644309,
 'eval_runtime': 27.9749,
 'eval_samples_per_second': 33.316,
 'eval_steps_per_second': 4.182,
 'epoch': 3.0}

In [ ]:

model.save_pretrained('/content/drive/MyDrive/fine_tuned_time_period_model')
tokenizer.save_pretrained('/content/drive/MyDrive/fine_tuned_time_period_model')

('/content/drive/MyDrive/fine_tuned_time_period_model/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_time_period_model/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_time_period_model/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_time_period_model/added_tokens.json')

In [ ]:
# I saved all the fine tuned models in my Drive

from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the fine-tuned intent model and tokenizer
intent_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/fine_tuned_intent_model1') #path file
intent_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/fine_tuned_intent_model1')

# Load the fine-tuned price type model and tokenizer
price_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/fine_tuned_price_type_model')
price_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/fine_tuned_price_type_model')

# Load the fine-tuned time period model and tokenizer
time_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/fine_tuned_time_period_model')
time_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/fine_tuned_time_period_model')
